In [ ]:
from qiskit import QuantumCircuit
from qiskit.visualization import plot_bloch_multivector, plot_state_qsphere
from qiskit.quantum_info import Statevector
from qiskit.primitives import StatevectorSampler
from random import getrandbits


RETAINED_CHAR = '_'

In [2]:
class CryptoCircuit(QuantumCircuit):
    def __init__(self, size=7):
        # Each qubit representing a bit in ASCII (7-bit)
        super().__init__(size)
        self.sampler = StatevectorSampler()

    def initialize(self, code:str):
        code = code[::-1] # reverse order
        for i in range(len(code)):
            self.x(i) if code[i]!='0' else self.id(i)

    def add_gate(self, gate:bool):
        self.h(self.qubits) if gate else self.id(self.qubits)

    def get_measurements(self):
        self.measure_all()
        result = self.sampler.run([(self)], shots=1).result()[0]
        counts = result.data['meas'].get_counts()
        return list(counts)[0]

    def visualize(self):
        statevector = Statevector(self)
        display(plot_bloch_multivector(statevector))
        # display(plot_state_qsphere(statevector))
    
    def draw(self, output=""):
        return ("You are not allowed to print the circuit..")


In [ ]:
class ProtocolBB84:
    
    def sender(message:str):
        circuits, gates = [], []
        for letter in message:
            # Skip the already found letters
            if letter == RETAINED_CHAR:
                circuits.append(None)
                gates.append(None)
                continue
        
            # Get letter's binary code
            letter = bin(ord(letter))[2:]

            # Create Quantum Circuit
            circuit = CryptoCircuit()

            # Alice's byte
            circuit.initialize(letter)

            # Choose a random sending basis
            gates.append(getrandbits(1))
            circuit.add_gate(gates[-1])

            # Save circuit and gate
            circuits.append(circuit)

        return circuits, gates
    
    def receiver(circuits:list[CryptoCircuit]):
        letters, gates = [], []
        for circuit in circuits:
            # Skip the already found letters
            if circuit == None:
                letters.append(RETAINED_CHAR)
                gates.append(None)
                continue

            # Choose a random receiving basis
            gates.append(getrandbits(1))
            circuit.add_gate(gates[-1])

            # Measure the qubits to get the received binary code
            letter = circuit.get_measurements()
            # Get letter from binary code
            letters.append(chr(int(letter,2)))

        return letters, gates

    def classicalChannel(sender_gates, receiver_gates):
        correct_chars = []
        for i, (sent,received) in enumerate(zip(sender_gates, receiver_gates)):
            if sent == received :
                correct_chars.append(i)
        return correct_chars


In [4]:
def main():
    message = "Hello, World! I am Myron Giannakis.."
    received_message = ""
    final_message = RETAINED_CHAR *len(message)

    # Loop until the message contains only the retained character
    while message.strip(RETAINED_CHAR) != '':
        # Sender
        circuits, sender_gates = ProtocolBB84.sender(message)
        # Receiver
        received_message, receiver_gates = ProtocolBB84.receiver(circuits)
        # Public discussion
        correct_chars = ProtocolBB84.classicalChannel(sender_gates, receiver_gates)

        # Process data
        received_correctly = ''.join([letter if i in correct_chars else RETAINED_CHAR
                                      for i,letter in enumerate(received_message)])

        message = ''.join([ letter if letter!=received_correctly[i] else RETAINED_CHAR
                           for i, letter in enumerate(message) ])
        
        print(message, '->', received_correctly)
        
        final_message = ''.join([ letter if letter!=RETAINED_CHAR else received_correctly[i]
                                 for i, letter in enumerate(final_message) ])

    
    print("\nFinal message received:", final_message, sep='\n')


if __name__=='__main__':
    main()



_e_l_,_W_rl____ a___y___ __a_____s._ -> H_l_o_ _o__d! I__m M_ron_Gi_nnaki__.
_e_l___W_______ a___y___ __a________ -> _____,___rl______________________s._
_e_l___W____________y______a________ -> _______________ a_______ ___________
_e__________________y______a________ -> ___l___W____________________________
___________________________a________ -> _e__________________y_______________
____________________________________ -> ___________________________a________

Final message received:
Hello, World! I am Myron Giannakis..
